In [28]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA as skPCA
from cuml import PCA as cumlPCA
import cudf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    train_rows = int(nrows*0.8) #We want to do an 80%/20% split for training and test data
    if os.path.exists(cached):
        print('use mortgage data')

        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]
        df_y_train = pd.DataFrame({'fea%d'%i:y[0:train_rows,i] for i in range(y.shape[1])})
        df_y_test = pd.DataFrame({'fea%d'%i:y[train_rows:,i] for i in range(y.shape[1])})

        df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
        df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})
        return df_X_train, df_X_test, df_y_train, df_y_test
    else:
        raise FileNotFoundError('Please download the required dataset or check the path')
   

In [4]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [5]:
%%time
nrows = 2**15
nrows = int(nrows * 1.5)
ncols = 400

X_train, X_test, y_train, y_test = load_data(nrows,ncols)
print('training data',X_train.shape)
print('training label',y_train.shape)
print('testing data',X_test.shape)
print('testing label',y_test.shape)

use mortgage data
training data (39321, 400)
training label (39321, 1)
testing data (9831, 400)
testing label (9831, 1)
CPU times: user 3.9 s, sys: 1.2 s, total: 5.1 s
Wall time: 5.09 s


In [6]:
n_components = 10
whiten = False
random_state = 42
svd_solver="full"

In [7]:
%%time
pca_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_sk = pca_sk.fit(X_train)

CPU times: user 2.61 s, sys: 287 ms, total: 2.9 s
Wall time: 702 ms


In [8]:
%%time
# Let's transform the test data
test_result_sk = pca_sk.transform(X_test)

CPU times: user 121 ms, sys: 9.58 ms, total: 131 ms
Wall time: 21.6 ms


In [9]:
print("We used SKL to dimensionally reduced our test data from ", X_test.shape, " to ", test_result_sk.shape)

We used SKL to dimensionally reduced our test data from  (9831, 400)  to  (9831, 10)


In [10]:
%%time
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf = y_train.values
y_cudf = y_cudf[:,0]
y_cudf = cudf.Series(y_cudf)

CPU times: user 1.44 s, sys: 7.84 ms, total: 1.45 s
Wall time: 1.44 s


In [11]:
%%time
pca_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_cuml = pca_cuml.fit(X_cudf)

CPU times: user 1.31 s, sys: 168 ms, total: 1.47 s
Wall time: 1.47 s


In [12]:
%%time
test_result_cuml = pca_cuml.transform(X_cudf_test)

CPU times: user 389 ms, sys: 183 µs, total: 389 ms
Wall time: 388 ms


In [13]:
print("We used cuML to dimensionally reduced our test data from ", X_cudf_test.shape, " to ", test_result_cuml.shape)

We used cuML to dimensionally reduced our test data from  (9831, 400)  to  (9831, 10)


In [24]:
pca_ft_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_ft_cuml= pca_ft_cuml.fit_transform(X_cudf)

In [25]:
pca_ft_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_ft_sk = pca_ft_sk.fit_transform(X_train)

In [30]:
t_result_ft_sk = pca_ft_sk.transform(X_test)
t_result_ft_cuml = pca_ft_cuml.transform(X_cudf_test)

In [31]:
passed = array_equal(t_result_ft_sk,t_result_ft_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results equal


In [26]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_ft_sk,attr),getattr(pca_ft_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn          singular_values_ equal
compare pca: cuml vs sklearn               components_ equal
compare pca: cuml vs sklearn       explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal


In [14]:
getattr(pca_sk,'components_')

array([[-0.00212977, -0.07055627,  0.1047976 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00723176,  0.01127368,  0.1313536 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09445507,  0.18742568, -0.52118821, ..., -0.        ,
        -0.        , -0.        ],
       ...,
       [-0.06312757,  0.06078236, -0.03260301, ..., -0.        ,
        -0.        , -0.        ],
       [-0.01253352, -0.00171504, -0.01075057, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01333629,  0.00361105, -0.01859554, ..., -0.        ,
        -0.        , -0.        ]])

In [15]:
print(getattr(pca_cuml,'components_'))

              0            1             2            3              4               5              6 ...  399
0 -0.0021280476 -0.070552796    0.10481354 -0.100896865     -0.9457314    0.0007198794 -1.2923238e-05 ...  0.0
1  0.0072329715  0.011286377    0.13132854 -0.036298826     0.29388025    0.0017393145 -1.8073086e-05 ...  0.0
2     0.0944582   0.18743177   -0.52120626     0.487186   -0.118696846  -2.3198118e-05  -8.768391e-07 ...  0.0
3    0.09282371   0.13502896   -0.37965295   0.38949713    -0.04821709   3.6900532e-05 -1.1448478e-06 ...  0.0
4  0.0018122958  0.008860328  0.0036243144  0.012946982   0.0033848248   6.4333726e-06  2.1142569e-08 ...  0.0
5   0.040010497 -0.024413597    0.03107638  0.054580383   0.0019069044    4.789955e-05   -4.90023e-07 ...  0.0
6    0.00852203  -0.01579137   0.027748648  0.012549743   -0.003918728   2.7484843e-05 -2.8291015e-07 ...  0.0
7   -0.06311759  0.060775872   -0.03258971  -0.02832294   0.0004889581    -6.27347e-05     8.9069e-07 ...  0.0
8

In [16]:
type(pca_sk.components_)

numpy.ndarray

In [17]:
type(pca_cuml.components_)

cudf.dataframe.dataframe.DataFrame

In [35]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_','mean_','noise_varaince_']:
    passed = array_equal(getattr(pca_sk,attr),getattr(pca_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn          singular_values_ equal
compare pca: cuml vs sklearn               components_ NOT equal
compare pca: cuml vs sklearn       explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal
compare pca: cuml vs sklearn                     mean_ equal


AttributeError: 'PCA' object has no attribute 'noise_varaince_'

In [19]:
passed = array_equal(result_sk,result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

TypeError: Expected sequence or array-like, got estimator PCA(copy=True, iterated_power='auto', n_components=10, random_state=42,
  svd_solver='full', tol=0.0, whiten=False)

In [20]:
passed = array_equal(test_result_sk,test_result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results NOT equal


In [21]:
test_result_cuml

<cudf.DataFrame ncols=10 nrows=9831 >

In [22]:
from sklearn.utils.testing import (assert_array_almost_equal, assert_equal)
assert_array_almost_equal(test_result_sk, to_nparray(test_result_cuml), decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

(mismatch 26.155019835215143%)
 x: array([-0.15, -0.03, -0.02, ..., -0.53,  0.7 ,  0.02])
 y: array([-0.15, -0.03, -0.02, ..., -0.53, -0.7 ,  0.02], dtype=float32)

In [23]:
!pip install matplotlib